In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import platform as pl
import plotly.express as px
import plotly.graph_objects as go


In [42]:
file = "logs_abdi_2.csv"
escavadeira = 24
caminhao = 25

sep = '\\' if pl.uname().system == 'Windows' else '/'
df = pd.read_csv(f"..{sep}dados_csv{sep}{file}", encoding='iso-8859-1')
df.loc[:,'date_created'] = pd.to_datetime(df['date_created'])


# Fator de Eficiência
O Fator de Eficiência leva em consideração os tempos efetivamente trabalhados pela escavadeira medido em uma hora, definindo sua relação em um indicador (E) adimensional. A periodicidade de aferição será ao final de cada turno de trabalho.
- E= Tempo efetivamente trabalhados (min) / 60 (min)   

In [4]:
df_data = df.loc[(df['imei'] == escavadeira)]
df_data.loc[:,'date_created'] = pd.to_datetime(df_data['date_created'])
df_data['last_date'] = df_data['date_created'].shift(1)
df_data['interval'] = (df_data['date_created'] - df_data['last_date']) / np.timedelta64(1, 'm')
df_data = df_data.set_index('date_created')
df_data = df_data.loc['2021-08-06':]

df_total = df_data.copy()
df_data = df_data.loc[df_data['weight'] > 0]
# Agrupa por hora
df_data = df_data.groupby(pd.Grouper(freq='3600s')).sum()
df_total = df_total.groupby(pd.Grouper(freq='3600s')).max()

df_total['eficiencia'] = df_data['interval']
df_total['eficiencia'].fillna(0, inplace=True)
print(df_total.loc[df_total['eficiencia'] > 0, ['weight', 'eficiencia']])
fig = px.bar(df_total, y="eficiencia", title="Fator de Eficiencia")
fig.show()

                      weight  eficiencia
date_created                            
2021-08-06 12:00:00   9390.0    1.266667
2021-08-18 06:00:00  10250.0    5.766667


## Rendimento

In [5]:
df_rend = df_total.copy()
# Agrupa por dia
df_rend = df_rend.groupby(pd.Grouper(freq='86400s')).sum()
fig = px.bar(df_rend, y="weight", title="Rendimento")
fig.show()

## Aferição do tempo de carga

In [6]:
import plotly.graph_objects as go

df_temp = df_data.copy()
df_temp.rename(columns = {'interval':'load_time'}, inplace = True)
df_temp = df_temp.loc[df_temp['weight'] > 0, ['weight', 'load_time']]
df_temp['weight'] = df_temp['weight']/1000

print(df_temp)
fig = go.Figure()
fig.add_trace(go.Bar(x=df_temp.index,
                y=df_temp['load_time'],
                name='Tempo em Min',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df_temp.index,
                y=df_temp['weight'],
                name='Carga em Ton.',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Aferição do tempo de Carga',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

                     weight  load_time
date_created                          
2021-08-06 12:00:00   93.26   1.266667
2021-08-18 06:00:00   41.30   5.766667


## Consumo de Combustível da Escavadeira

In [7]:
df_comb = df_total.copy()
litragem = 345
df_max = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
df_min = df_comb.groupby(pd.Grouper(freq='86400s')).min().fillna(0)
df_comb = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
diff_fuel = df_max['fuel'] - df_min['fuel']
df_comb['diff_fuel'] = diff_fuel
df_comb['consumo'] = (diff_fuel*litragem)/100
# print(df_comb.loc[:, ['diff_fuel', 'consumo']])

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['diff_fuel'],
                name='Nivel Combustivel (%)',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['consumo'],
                name='Consumo em Litros',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Consumo de Combustível da Escavadeira',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Consumo de Combustível do Caminhão

Dados do Caminhão:
 - Modelo: Mercedes 2423k (mb 2423)
 - Bateria (qtde x Ah/V): 2 x 135/12
 - Volume Caçamba: 12 m³
 - Velocidade Máxima: 82 Km/h
 - Capacidade Subida (em mov.): 55% * 23t
 - Capacidade Subida (parado): 39% * 23t
 - Carga Máxima útil: 16,050 t
 - Capacidade Combustível: 210 L

In [8]:
df_data = df.loc[(df['imei'] == caminhao)]
df_data.loc[:,'date_created'] = pd.to_datetime(df_data['date_created'])
df_data = df_data.set_index('date_created')
df_data = df_data.loc['2021-08-06':]

df_comb = df_data.copy()
litragem = 300
df_max = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
df_min = df_comb.groupby(pd.Grouper(freq='86400s')).min().fillna(0)
df_comb = df_comb.groupby(pd.Grouper(freq='86400s')).max().fillna(0)
diff_fuel = df_max['fuel'] - df_min['fuel']
df_comb['diff_fuel'] = diff_fuel
df_comb['consumo'] = (diff_fuel*litragem)/100
# print(df_comb.loc[:, ['diff_fuel', 'consumo']])

fig = go.Figure()
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['diff_fuel'],
                name='Consumo de Combustivel (%)',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=df_comb.index,
                y=df_comb['consumo'],
                name='Consumo em Litros',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Consumo de Combustível do Caminhão',
    xaxis_tickfont_size=14,
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()

## Peso Transportado por Combustível Consumido

Obs: Uma vez que os caminhões não possuem sensores para aferição de carga transportada, este indicador se torna inviável de ser mensurado.

## Trajeto Ciclo de Carga

In [3]:
# Plota Gráfico
def plotDatas(fig, datas, is_line=True, label=''):
        if fig is None:
            fig = go.Figure(go.Scattermapbox(
            mode = "markers+lines" if is_line == True else "markers",
            lon = datas[:,1],
            lat = datas[:,0],
            text = label if label != '' else 'veic1',
            marker = {'size': 10,
                    'color': 'blue'}))
        else:
            fig.add_trace(go.Scattermapbox(
            mode = "markers+lines" if is_line == True else "markers",
            lon = datas[:,1],
            lat = datas[:,0],
            text = label if label != '' else 'veic2',
            marker = {'size': 10,
                    'color': 'red'}))    

        fig.update_layout(
            margin ={'l':0,'t':0,'b':0,'r':0},
            #'center': {'lon': -38.71901, 'lat':  -3.94467},
            mapbox = {
                'center': {'lon': datas[:,1].mean(), 'lat':  datas[:,0].mean()},
                'style': "stamen-terrain",
                'center': {'lon': datas[:,1].mean(), 'lat':  datas[:,0].mean()},
                'zoom': 16})
        return fig

# Geração de Dados
### Trajeto do Caminhão dos dias 05/08 a 18/08

In [49]:
# import gmaps
from datetime import datetime
df_interval = df.loc[(df['imei'] == caminhao)]
df_gps = df.loc[:, ['latitude','longitude']]
gps_tuples = [tuple(x) for x in df_gps.to_numpy()]
start = gps_tuples[0]
end = gps_tuples[1]
# api_key_google = open('../api-google.txt').read()
# gmaps.configure(api_key=api_key_google)
# fig = gmaps.figure()
# layer = gmaps.directions.Directions(start, end,waypoints=gps_tuples)
# markers = gmaps.marker_layer(gps_tuples)
# layer = gmaps.symbol_layer(df_gps, fill_color = 'red', stroke_color='red', scale=2)
# fig.add_layer(layer)
# fig
def plotDataTimestamp(df):
    subset = df.loc[:, ['date_created','velocity','fuel']]
    text_df = [tuple(x) for x in subset.to_numpy()]
    fig = go.Figure(go.Scattermapbox(
        lat = df['latitude'].to_numpy(),
        lon = df['longitude'].to_numpy(),
        mode = 'markers',
        marker = go.scattermapbox.Marker( size = 9),
        # text = df['date_created'].astype('string').to_numpy()
        text = text_df
    ))

    mapbox_access_token = open('../mapbox_token.txt').read()

    fig.update_layout(
        margin ={'l':0,'t':0,'b':0,'r':0},
        #'center': {'lon': -38.71901, 'lat':  -3.94467},
        mapbox = {
            'center': {'lon': df['longitude'].to_numpy()[0], 'lat':  df['latitude'].to_numpy()[0]},
            'style': "stamen-terrain",
            'center': {'lon': df['longitude'].to_numpy()[0], 'lat':  df['latitude'].to_numpy()[0]},
            'zoom': 16})

    fig.show()


plotDataTimestamp(df_interval)



### Trajeto de Abastecimento

In [68]:
from geopy.distance import great_circle

df_abast = df.copy()
df_abast = df_abast.loc[(df['imei'] == caminhao) & 
                        (df['date_created'] >= '2021-08-07 12:29:18') &
                        (df['date_created'] <= '2021-08-07 12:41:47')]


timestamp = df_abast['date_created'].to_numpy()
interval = float((timestamp[-1] - timestamp[0]) / (10**9))
location = df_abast.loc[:,['latitude','longitude']].to_numpy()
distance = great_circle(location[0], location[-1]).m
vel_media = (distance*3600)/(1000*interval)
print(f'Total Distance: {round(distance, 2)} m, Interval: {round((interval / 60), 2)} min, Vel Media: {round(vel_media, 2)} Km/h')
plotDataTimestamp(df_abast)

Total Distance: 3920.73 m, Interval: 12.48 min, Vel Media: 18.84 Km/h


### Trajeto Ponto de Apoio 


In [95]:
from geopy.distance import great_circle
from IPython.display import display, HTML
import random

litragem_cam = 210
df_apoio = df.copy()
df_apoio = df_apoio.loc[(df['imei'] == caminhao) & 
                        (df['date_created'] >= '2021-08-07 08:50:23') &
                        (df['date_created'] <= '2021-08-07 08:55:50')]


timestamp = df_apoio['date_created'].to_numpy()
interval = float((timestamp[-1] - timestamp[0]) / (10**9))
location = df_apoio.loc[:,['latitude','longitude']].to_numpy()
distance = great_circle(location[0], location[-1]).m
vel_media = (distance*3600)/(1000*interval)
autonomia = (distance/1000)/(0.71*litragem_cam/100)
print(f'Total Distance: {round(distance, 2)} m, Interval: {round((interval / 60), 2)} min, \
        Vel Media: {round(vel_media, 2)} Km/h, Autonomia: {round(autonomia, 2)} Km/L, max: {int(df["fuel"].max())}, min: {int(df["fuel"].min())}')

last_lat = df_apoio['latitude'].to_numpy()[0]
last_lon = df_apoio['longitude'].to_numpy()[0]

def calcDistance(df):
        global last_lat, last_lon
        dist = great_circle([df['latitude'],df['longitude']], [last_lat, last_lon]).m
        last_lat = df['latitude']
        last_lon = df['longitude']
        return dist

df_apoio['distance'] = df_apoio.apply(lambda x: calcDistance(x), axis=1)

nivel_comb = df_apoio['fuel'].to_numpy()[0]*litragem_cam/100
def calcLevel(df):
        global nivel_comb
        autonomia = random.uniform(1.6, 2.4)
        nivel_calc = (nivel_comb - ((df['distance']/1000)/autonomia))
        nivel_comb = nivel_calc
        return nivel_calc


df_apoio['nivel_litros'] = df_apoio.apply(lambda x: calcLevel(x), axis=1)
df_apoio['fuel_est'] = df_apoio.apply(lambda x: int(x['nivel_litros']*100/litragem_cam), axis=1)
display(df_apoio.head())
plotDataTimestamp(df_apoio)

Total Distance: 3281.38 m, Interval: 5.45 min,         Vel Media: 36.13 Km/h, Autonomia: 2.2 Km/L, max: 114, min: 5


,latitude,longitude,status,angle,fuel,weight,velocity,imei,date_created,distance,nivel_litros,fuel_est
6622,-3.879148,-38.761066,Parado,2.375795,96.0,-1.0,0.747417,25,2021-08-07 08:50:23,0.000000,201.600000,96
6623,-3.879255,-38.761116,Movimento,-7.255507,95.0,-1.0,16.074614,25,2021-08-07 08:50:28,13.127406,201.592926,95
6624,-3.880620,-38.761467,Movimento,6.199142,92.0,-1.0,47.606140,25,2021-08-07 08:50:54,156.696793,201.497159,95
6625,-3.883064,-38.761730,Parado,6.118005,92.0,-1.0,0.444744,25,2021-08-07 08:51:25,273.322582,201.346424,95
6626,-3.883070,-38.761730,Parado,6.175217,93.0,-1.0,0.245021,25,2021-08-07 08:51:31,0.667171,201.346120,95


### Obtem todas as posições geográficas entre dois pontos

In [41]:
from geopy.distance import great_circle

start_point = (-3.879148, -38.761066)
end_point = (-3.908592, -38.759087)

df_point = df.copy()
df_point = df_point.loc[(df['imei'] == caminhao) & 
                        (df['latitude'] >= start_point[0]) & 
                        (df['latitude'] <= end_point[0])]

# timestamp = df_apoio['date_created'].to_numpy()
# interval = float((timestamp[-1] - timestamp[0]) / (10**9))
# location = df_apoio.loc[:,['latitude','longitude']].to_numpy()
# distance = great_circle(location[0], location[-1]).m
# vel_media = (distance*3600)/(1000*interval)
# print(f'Total Distance: {round(distance, 2)} m, Interval: {round((interval / 60), 2)} min, Vel Media: {round(vel_media, 2)} Km/h')
plotDataTimestamp(df_apoio)